In [3]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from tensorflow.keras.preprocessing import image


model = model_from_json(open("fer.json", "r").read())
model.load_weights('fer.h5')


face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture('InShot_20210108_003820121_1.mp4')
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('Face_Emotion.mp4',fourcc,20.0,(856,480))


while True:
    ret,test_img=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(0,0,0),thickness=2)
        roi_gray=test_img[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)[0]

        #find max indexed array
        #max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        #predicted_emotion = np.max[predictions]
        
        for (i, (emotion, prob)) in enumerate(zip(emotions, predictions)):
                text = "{}: {:.2f}%".format(emotion, prob * 100)                
                w = int(prob * 300)
                cv2.putText(test_img, text, (int(x+h), int(y+i*10)), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1)

    resized_img = cv2.resize(test_img, (856, 480))
    #cv2.imshow('Facial emotion analysis ',resized_img)
    out.write(resized_img)


    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows


KeyboardInterrupt: 